In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
cols_demos = [
    'MatchId',
    'MapName' ,
    'WinnerId' ,
    'WinnerScore' ,
    'WinnerFirstHalfScore' ,
    'WinnerSecondHalfScore' ,
    'WinnerFirstHalfSide' ,
    'WinnerOTScore' ,
    'LoserId' ,
    'LoserScore' ,
    'LoserFirstHalfScore' ,
    'LoserSecondHalfScore' ,
    'LoserFirstHalfSide' ,
    'LoserOTScore' ,
    'DemoParsed' ,
    'Created' ,
    'Updated']
cols_map_picks = [
    'MatchId',
    'MapName',
    'DecisionOrder',
    'DecisionTeamId',
    'OtherTeamId',
    'Decision',
    'Created',
    'Updated']
cols_matches = [
    'MatchId',
    'HLTVMatchId',
    'CompetitionId',
    'HLTVLink',
    'MatchType',
    'MatchDate',
    'MatchTime',
    'Stars',
    'Slug',
    'WinnerId',
    'WinnerScore',
    'LoserId',
    'LoserScore',
    'Created',
    'Updated']
cols_teams = [
    'TeamId',
    'HLTVTeamId',
    'HLTVLink',
    'TeamName',
    'Country',
    'Twitter',
    'Facebook',
    'Created',
    'Updated']
cols_player_demos = [
    'MatchId',
    'PlayerId',
    'TeamId',
    'MapName',
    'Side',
    'Kills',
    'Deaths',
    'ADR',
    'KAST',
    'HLTVRating',
    'Created',
    'Updated']
cols_players = [
    'PlayerId',
    'HLTVPlayerId',
    'HLTVLink',
    'Country',
    'RealName',
    'PlayerName',
    'Facebook',
    'Twitter',
    'Twitch',
    'Created',
    'Updated']
data_dir = '../data/'
demos = pd.read_csv(data_dir+'demos.csv',names = cols_demos)
map_picks = pd.read_csv(data_dir+'map_picks.csv',names = cols_map_picks)
matches = pd.read_csv(data_dir+'matches.csv',names = cols_matches)
teams = pd.read_csv(data_dir+'teams.csv',names = cols_teams)
players = pd.read_csv(data_dir+'players.csv',names = cols_players)
player_demos = pd.read_csv(data_dir+'player_demos.csv',names = cols_player_demos)

In [3]:
def build_team_record_df(teams,demos):
    '''Creates team_record dataframe where each row is a Team, columns are various stats.
       Inputs are dataframes of the .csvs of the same name.
       Output is a dataframe indexed by teamid'''
    # All team IDs
    team_record = pd.DataFrame(teams['TeamId'].unique().astype(int),columns=['TeamId'])
    # Wins and losses (For games, not matches) by team ID
    game_wins = demos['WinnerId'].value_counts()
    game_losses = demos['LoserId'].value_counts()
    # merge wins and losses 
    team_record = team_record.merge(game_wins,left_on='TeamId',right_index=True,how='left').fillna(0).astype(int)
    team_record = team_record.merge(game_losses,left_on='TeamId',right_index=True,how='left').fillna(0).astype(int)
    team_record.rename(columns={"WinnerId": "GameWins", "LoserId": "GameLosses"},inplace=True)
    team_record['TotalGames'] = team_record['GameWins']+team_record['GameLosses'].astype(int)
    team_record['WinPercent'] = (team_record['GameWins']/team_record['TotalGames']).fillna(0)
    # Set index to team_id, to simplify things
    team_record.set_index('TeamId',drop=False,inplace=True)
    return team_record    

def win_pct_by_map(demos,team_record):
    '''Calculates win percentage by map for teams in team_record 
       and appends win/loss/matches/win_pct columns to team_record.
       demos - the demos dataframe, no alterations
       team_record - output of build_team_record_df, or a filtered version (i.e. removing few-match teams)
       '''
    
    # Count wins and losses by map and by team
    wins = demos.groupby(['WinnerId','MapName']).count()['MatchId']
    losses = demos.groupby(['LoserId','MapName']).count()['MatchId']
    
    # Fills the dataframe with wins and losses, sums for matches, divides for win_pct
    for m in demos['MapName'].unique():
        for t in team_record['TeamId']:
            # Wins
            try:
                team_record.loc[t,m+'_wins'] = wins[(t,m)]
            except KeyError:
                team_record.loc[t,m+'_wins'] = 0
            # Losses
            try:
                team_record.loc[t,m+'_losses'] = losses[(t,m)]
            except KeyError:
                team_record.loc[t,m+'_losses'] = 0
        team_record[m+'_games'] = team_record[m+'_wins'] + team_record[m+'_losses']
        # Map level win percentage
        team_record[m+'_win_pct'] = team_record[m+'_wins'] / team_record[m+'_games']

    return team_record

In [4]:
def remove_records(df,cols,remove_ids):
    # Function removes rows where df[cols] value is in remove_ids
    # Returns df with same columns and fewer rows

    few_games_dict = {tid: [] for tid in cols}
    out = [False]*df.shape[0]
    for col in cols:
        a = [t in remove_ids for t in df[col]]
        out = [a or b for (a,b) in zip(out,a)]
    df['remove'] = out
    print("Num removed: ",df['remove'].sum())
    # Remove bad records
    df_out = df[df['remove']==False]
    df_out = df_out.drop(['remove'],axis=1)

    return df_out

### Matches without map_picks data
26 maps in matches.csv do not have map_pick records. 24 of those are also in demos.csv. Removing those.

In [5]:
matches_match_ids = matches['MatchId'].unique()
map_pick_match_ids = map_picks['MatchId'].unique()
demos_match_ids = demos['MatchId'].unique()
no_map_picks = []

for m in matches_match_ids:
    if m not in map_pick_match_ids:
        no_map_picks.append(m)

for m in demos_match_ids:
    if m not in map_pick_match_ids:
        no_map_picks.append(m)

no_map_picks = set(no_map_picks)
print(len(no_map_picks))

26


In [6]:
demos = remove_records(demos,['MatchId'],no_map_picks)
player_demos = remove_records(player_demos,['MatchId'],no_map_picks)
matches = remove_records(matches,['MatchId'],no_map_picks)

Num removed:  41
Num removed:  0
Num removed:  26


In [7]:
print(teams.shape)
print(map_picks.shape)
print(matches.shape)
print(demos.shape)
print(player_demos.shape)

(636, 9)
(43705, 8)
(6257, 15)
(13113, 17)
(392640, 12)


### Remove Weird other maps
Three maps only appear about 5 times, removing them. Note: I guess they got cleared out in a previous filter.

In [8]:
weird_maps = ['de_tuscan','de_cobblestone','de_cache']

In [9]:
# Label each record with T/F if a team involved has fewer than min_games
demos = remove_records(demos,['MapName'],weird_maps)
map_picks = remove_records(map_picks,['MapName'],weird_maps)
player_demos = remove_records(player_demos,['MapName'],weird_maps)

Num removed:  0
Num removed:  0
Num removed:  0


In [10]:
print(teams.shape)
print(map_picks.shape)
print(matches.shape)
print(demos.shape)
print(player_demos.shape)

(636, 9)
(43705, 8)
(6257, 15)
(13113, 17)
(392640, 12)


### Remove matches where Decision 1 was a Pick
This happens ~3 times

In [11]:
wrong_decision_match_ids = list(map_picks[(map_picks['Decision']=='Pick') & (map_picks['DecisionOrder']==1)]['MatchId'])

In [12]:
demos = remove_records(demos,['MatchId'],wrong_decision_match_ids)
player_demos = remove_records(player_demos,['MatchId'],wrong_decision_match_ids)
matches = remove_records(matches,['MatchId'],wrong_decision_match_ids)

Num removed:  8
Num removed:  240
Num removed:  3


In [13]:
print(teams.shape)
print(map_picks.shape)
print(matches.shape)
print(demos.shape)
print(player_demos.shape)

(636, 9)
(43705, 8)
(6254, 15)
(13105, 17)
(392400, 12)


### Teams with fewer than min_games
After the other filters.

In [14]:
# Team Ids for teams with fewer than 25 games
min_games = 25

# Repeat to remove teams that fall below 25 games when others are removed
for i in range(5):
    print("Round ",i)
    team_record = build_team_record_df(teams,demos)
    few_games_team_ids = team_record[team_record['TotalGames']<min_games]['TeamId']
    # Label each record with T/F if a team involved has fewer than min_games
    teams = remove_records(teams,['TeamId'],few_games_team_ids)
    demos = remove_records(demos,['WinnerId','LoserId'],few_games_team_ids)
    map_picks = remove_records(map_picks,['DecisionTeamId','OtherTeamId'],few_games_team_ids)
    player_demos = remove_records(player_demos,['TeamId'],few_games_team_ids)

Round  0
Num removed:  427
Num removed:  2431
Num removed:  9008
Num removed:  43981
Round  1
Num removed:  25
Num removed:  434
Num removed:  1819
Num removed:  11756
Round  2
Num removed:  4
Num removed:  77
Num removed:  315
Num removed:  2180
Round  3
Num removed:  0
Num removed:  0
Num removed:  0
Num removed:  0
Round  4
Num removed:  0
Num removed:  0
Num removed:  0
Num removed:  0


In [15]:
print(teams.shape)
print(map_picks.shape)
print(matches.shape)
print(demos.shape)
print(player_demos.shape)

(180, 9)
(32563, 8)
(6254, 15)
(10163, 17)
(334483, 12)


### Repeat: Matches without map_picks data
26 maps in matches.csv do not have map_pick records. 24 of those are also in demos.csv. Removing those.
Repeating here because more dangling matches get removed

In [16]:
matches_match_ids = matches['MatchId'].unique()
map_pick_match_ids = map_picks['MatchId'].unique()
demos_match_ids = demos['MatchId'].unique()
no_map_picks = []

for m in matches_match_ids:
    if m not in map_pick_match_ids:
        no_map_picks.append(m)

for m in demos_match_ids:
    if m not in map_pick_match_ids:
        no_map_picks.append(m)

no_map_picks = set(no_map_picks)
print(len(no_map_picks))

1596


In [17]:
demos = remove_records(demos,['MatchId'],no_map_picks)
player_demos = remove_records(player_demos,['MatchId'],no_map_picks)
matches = remove_records(matches,['MatchId'],no_map_picks)

Num removed:  0
Num removed:  30103
Num removed:  1596


In [18]:
print(teams.shape)
print(map_picks.shape)
print(matches.shape)
print(demos.shape)
print(player_demos.shape)

(180, 9)
(32563, 8)
(4658, 15)
(10163, 17)
(304380, 12)


### Save clean csv's

In [19]:
out_dir = '../data/clean/'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [20]:
demos.to_csv(out_dir+'demos.csv',index=False)
map_picks.to_csv(out_dir+'map_picks.csv',index=False)
matches.to_csv(out_dir+'matches.csv',index=False)
teams.to_csv(out_dir+'teams.csv',index=False)
players.to_csv(out_dir+'players.csv',index=False)
player_demos.to_csv(out_dir+'player_demos.csv',index=False)